In [32]:
import pandas as pd
import numpy as np
import xarray as xr
import warnings
from itertools import groupby
warnings.filterwarnings('ignore')

In [33]:
data_workshop=pd.read_csv('/data/raw/Workshop_PUSO.csv',sep=';')

In [3]:
data_workshop.query('is_puso==1').groupby(['tahun','bulan'])['idsegmen'].agg('count')

tahun  bulan
22     1         47
       2         32
       3         13
       4         21
       5         14
       6         27
       7         13
       8         25
       9         19
       10         3
       11         5
       12        12
23     1          6
       2          6
       3          8
       4          3
       5         14
       6         16
       7         68
       8         85
       9         82
       10       109
       11        43
       12        13
Name: idsegmen, dtype: int64

In [4]:
data_workshop=data_workshop.sort_values(['tahun','bulan'])

In [5]:
for g in ['mean','max','min']:
    for h in ['total_precipitation_0_daily','2m_temperature_0_daily','2m_dewpoint_temperature_0_daily',
          'convective_rain_rate_0_daily','potential_evaporation_0_daily','evaporation_0_daily','soil_temperature_level_1_0_daily',
         'volumetric_soil_water_layer_1_0_daily']:
        nc_list=[]
        for i in [20,21,22,23]:
            for j in range(1,13):
                temp=data_workshop.query('tahun==@i and bulan==@j')
                periode='20'+str(i)+'_'+str(j).zfill(2)
                if g=='mean':
                    netcdf_file = f"/data/ksa/00_Code/12_Workshop_Puso/climate_data/{periode}_data/{h}-{g}.nc"
                else:
                    netcdf_file = f"/data/ksa/00_Code/12_Workshop_Puso/climate_data/{periode}_data_{g}/{h}-{g}.nc"
                dt=xr.open_dataset(netcdf_file)
                nc_list.append(dt)
        combined = xr.concat(nc_list, dim="valid_time")
        combined.to_netcdf(f"/data/ksa/00_Code/12_Workshop_Puso/concated_climate_data/concated-ERA5-{h}-{g}.nc")

In [6]:
import calendar
from datetime import datetime, timedelta
def get_last_day_of_month(year, month):
    _, last_day = calendar.monthrange(year, month)
    return last_day
def days_before(date_str, days):
    # Parse the input date (format: YYYY-MM-DD)
    date = datetime.strptime(date_str, "%Y-%m-%d")
    # Subtract the number of days
    new_date = date - timedelta(days=days)
    return new_date.strftime("%Y-%m-%d")  # Format the result as a string

In [7]:
dt_x=data_workshop.query('tahun==22').query('bulan==1')

In [8]:
dict_nc_value={
    'total_precipitation_0_daily':'tp',
    '2m_temperature_0_daily':'t2m',
    '2m_dewpoint_temperature_0_daily':'d2m',
    'convective_rain_rate_0_daily':'crr',
    'potential_evaporation_0_daily':'pev',
    'evaporation_0_daily':'e',
    'soil_temperature_level_1_0_daily':'stl1',
    'volumetric_soil_water_layer_1_0_daily':'swvl1'
}

In [20]:
data_compile=pd.DataFrame()
print('RUNNING_DATA=================')
print('***************************')
for i in data_workshop.tahun.unique():
    print('Tahun:',i)
    for j in data_workshop.bulan.unique():
        print('Bulan:',j)
        temp_df_period=data_workshop.query('tahun==@i').query('bulan==@j')
        for k in ['total_precipitation_0_daily','2m_temperature_0_daily','2m_dewpoint_temperature_0_daily',
                  'convective_rain_rate_0_daily','potential_evaporation_0_daily','evaporation_0_daily',
                  'soil_temperature_level_1_0_daily','volumetric_soil_water_layer_1_0_daily']:
            print('Data:',k)
            print('Daily Statistics:')
            for l in ['mean','max','min']:
                print(l,end ='|')
                last_day=get_last_day_of_month(int('20'+str(i)),j)
                last_date=f'20{i}-{str(j).zfill(2)}-{last_day}'
                date_before=days_before(last_date,719)
                ncfile=xr.open_dataset(f'/data/ksa/00_Code/12_Workshop_Puso/concated_climate_data/concated-ERA5-{k}-{l}.nc')
                keys=dict_nc_value[k]
                var_nc=f'{keys}_{l}'
                temp_df_period[var_nc]=temp_df_period.apply(lambda y: ncfile[keys]
                                                                   .sel(latitude=y['latitude'], 
                                                                        longitude=y['longitude'], 
                                                                        method='nearest')  # Select nearest latitude and longitude
                                                                   .sel(valid_time=slice(date_before, last_date))  
                                                                   .values, axis=1)
                list_var=[f'{keys}_{l}_{59-o}' for o in range(0,60)]
                temp_df_period[var_nc]=temp_df_period[var_nc].apply(lambda y:y.reshape(-1,12).mean(axis=1))#.values)
                for m in range(0,60):
                    temp_df_period[f'ERA5_{keys}_{l}_{(str(59-m).zfill(2))}']=temp_df_period[var_nc].apply(lambda y:y[m])
                temp_df_period.drop(var_nc,axis=1,inplace=True)
            print('\n')
        data_compile=pd.concat([temp_df_period,data_compile])
        print('***************************')

RUNNING_DATA=================
***************************
Tahun: 22
Bulan: 1
Data: total_precipitation_0_daily
Daily Statistics:
mean|max|min|

Data: 2m_temperature_0_daily
Daily Statistics:
mean|max|min|

Data: 2m_dewpoint_temperature_0_daily
Daily Statistics:
mean|max|min|

Data: convective_rain_rate_0_daily
Daily Statistics:
mean|max|min|

Data: potential_evaporation_0_daily
Daily Statistics:
mean|max|min|

Data: evaporation_0_daily
Daily Statistics:
mean|max|min|

Data: soil_temperature_level_1_0_daily
Daily Statistics:
mean|max|min|

Data: volumetric_soil_water_layer_1_0_daily
Daily Statistics:
mean|max|min|

***************************
Bulan: 2
Data: total_precipitation_0_daily
Daily Statistics:
mean|max|min|

Data: 2m_temperature_0_daily
Daily Statistics:
mean|max|min|

Data: 2m_dewpoint_temperature_0_daily
Daily Statistics:
mean|max|min|

Data: convective_rain_rate_0_daily
Daily Statistics:
mean|max|min|

Data: potential_evaporation_0_daily
Daily Statistics:
mean|max|min|

Data

In [ ]:
data_compile.drop(['latitude','longitude'],axis=1).to_csv('data_compile_agro_ERA5 data.csv',sep=';',index=False)